# Setup

In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from joblib import dump
import streamlit as st
from scipy import stats
import yaml, time, sys, os, glob

import seaborn as sns
sns.set_style("darkgrid")

from IPython.display import display, Markdown
pd.set_option('display.max_columns', None)  

DATASET = "Texas_Inpatient_Discharge"
SPLIT_TRAINING = True
DEBUG = False
SEED = 42

COLAB = 'google.colab' in sys.modules
if COLAB:
    ROOT = f"/content/gdrive/MyDrive/datasets/{DATASET.replace(' ','_')}/"
else:
    ROOT = "./"

In [3]:
if COLAB:
  from google.colab import drive
  if not os.path.isdir("/content/gdrive"):
    drive.mount("/content/gdrive")
    d = "/content/gdrive/MyDrive/datasets"
    if not os.path.isdir(d): os.makedirs(d)
  if not os.path.isdir(ROOT): os.makedirs(ROOT)

def makedirs(d):
  if COLAB:
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d)
  else:
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d, mode=0o777, exist_ok=True)

for d in ['doc','orig','data','output']: makedirs(d)

# Dataset

In [4]:
df = pd.read_pickle(f"{ROOT}/data/df_train_sample_00_of_1_clean.pkl")
print(df.shape)
df.head()

(999698, 195)


,RECORD_ID,DISCHARGE,THCIC_ID,PROVIDER_NAME,TYPE_OF_ADMISSION,SOURCE_OF_ADMISSION,SPEC_UNIT_1,SPEC_UNIT_2,SPEC_UNIT_3,SPEC_UNIT_4,SPEC_UNIT_5,PAT_STATE,PAT_ZIP,PAT_COUNTRY,COUNTY,PUBLIC_HEALTH_REGION,PAT_STATUS,SEX_CODE,RACE,ETHNICITY,ADMIT_WEEKDAY,LENGTH_OF_STAY,PAT_AGE,FIRST_PAYMENT_SRC,SECONDARY_PAYMENT_SRC,TYPE_OF_BILL,TOTAL_CHARGES,TOTAL_NON_COV_CHARGES,TOTAL_CHARGES_ACCOMM,TOTAL_NON_COV_CHARGES_ACCOMM,TOTAL_CHARGES_ANCIL,TOTAL_NON_COV_CHARGES_ANCIL,POA_PROVIDER_INDICATOR,ADMITTING_DIAGNOSIS,PRINC_DIAG_CODE,POA_PRINC_DIAG_CODE,OTH_DIAG_CODE_1,POA_OTH_DIAG_CODE_1,OTH_DIAG_CODE_2,POA_OTH_DIAG_CODE_2,OTH_DIAG_CODE_3,POA_OTH_DIAG_CODE_3,OTH_DIAG_CODE_4,POA_OTH_DIAG_CODE_4,OTH_DIAG_CODE_5,POA_OTH_DIAG_CODE_5,OTH_DIAG_CODE_6,POA_OTH_DIAG_CODE_6,OTH_DIAG_CODE_7,POA_OTH_DIAG_CODE_7,OTH_DIAG_CODE_8,POA_OTH_DIAG_CODE_8,OTH_DIAG_CODE_9,POA_OTH_DIAG_CODE_9,OTH_DIAG_CODE_10,POA_OTH_DIAG_CODE_10,OTH_DIAG_CODE_11,POA_OTH_DIAG_CODE_11,OTH_DIAG_CODE_12,POA_OTH_DIAG_CODE_12,OTH_DIAG_CODE_13,POA_OTH_DIAG_CODE_13,OTH_DIAG_CODE_14,POA_OTH_DIAG_CODE_14,OTH_DIAG_CODE_15,POA_OTH_DIAG_CODE_15,OTH_DIAG_CODE_16,POA_OTH_DIAG_CODE_16,OTH_DIAG_CODE_17,POA_OTH_DIAG_CODE_17,OTH_DIAG_CODE_18,POA_OTH_DIAG_CODE_18,OTH_DIAG_CODE_19,POA_OTH_DIAG_CODE_19,OTH_DIAG_CODE_20,POA_OTH_DIAG_CODE_20,OTH_DIAG_CODE_21,POA_OTH_DIAG_CODE_21,OTH_DIAG_CODE_22,POA_OTH_DIAG_CODE_22,OTH_DIAG_CODE_23,POA_OTH_DIAG_CODE_23,OTH_DIAG_CODE_24,POA_OTH_DIAG_CODE_24,E_CODE_1,POA_E_CODE_1,E_CODE_2,POA_E_CODE_2,E_CODE_3,POA_E_CODE_3,E_CODE_4,POA_E_CODE_4,E_CODE_5,POA_E_CODE_5,E_CODE_6,POA_E_CODE_6,E_CODE_7,POA_E_CODE_7,E_CODE_8,POA_E_CODE_8,E_CODE_9,POA_E_CODE_9,E_CODE_10,POA_E_CODE_10,PRINC_SURG_PROC_CODE,PRINC_SURG_PROC_DAY,PRINC_ICD9_CODE,OTH_SURG_PROC_CODE_1,OTH_SURG_PROC_DAY_1,OTH_ICD9_CODE_1,OTH_SURG_PROC_CODE_2,OTH_SURG_PROC_DAY_2,OTH_ICD9_CODE_2,OTH_SURG_PROC_CODE_3,OTH_SURG_PROC_DAY_3,OTH_ICD9_CODE_3,OTH_SURG_PROC_CODE_4,OTH_SURG_PROC_DAY_4,OTH_ICD9_CODE_4,OTH_SURG_PROC_CODE_5,OTH_SURG_PROC_DAY_5,OTH_ICD9_CODE_5,OTH_SURG_PROC_CODE_6,OTH_SURG_PROC_DAY_6,OTH_ICD9_CODE_6,OTH_SURG_PROC_CODE_7,OTH_SURG_PROC_DAY_7,OTH_ICD9_CODE_7,OTH_SURG_PROC_CODE_8,OTH_SURG_PROC_DAY_8,OTH_ICD9_CODE_8,OTH_SURG_PROC_CODE_9,OTH_SURG_PROC_DAY_9,OTH_ICD9_CODE_9,OTH_SURG_PROC_CODE_10,OTH_SURG_PROC_DAY_10,OTH_ICD9_CODE_10,OTH_SURG_PROC_CODE_11,OTH_SURG_PROC_DAY_11,OTH_ICD9_CODE_11,OTH_SURG_PROC_CODE_12,OTH_SURG_PROC_DAY_12,OTH_ICD9_CODE_12,OTH_SURG_PROC_CODE_13,OTH_SURG_PROC_DAY_13,OTH_ICD9_CODE_13,OTH_SURG_PROC_CODE_14,OTH_SURG_PROC_DAY_14,OTH_ICD9_CODE_14,OTH_SURG_PROC_CODE_15,OTH_SURG_PROC_DAY_15,OTH_ICD9_CODE_15,OTH_SURG_PROC_CODE_16,OTH_SURG_PROC_DAY_16,OTH_ICD9_CODE_16,OTH_SURG_PROC_CODE_17,OTH_SURG_PROC_DAY_17,OTH_ICD9_CODE_17,OTH_SURG_PROC_CODE_18,OTH_SURG_PROC_DAY_18,OTH_ICD9_CODE_18,OTH_SURG_PROC_CODE_19,OTH_SURG_PROC_DAY_19,OTH_ICD9_CODE_19,OTH_SURG_PROC_CODE_20,OTH_SURG_PROC_DAY_20,OTH_ICD9_CODE_20,OTH_SURG_PROC_CODE_21,OTH_SURG_PROC_DAY_21,OTH_ICD9_CODE_21,OTH_SURG_PROC_CODE_22,OTH_SURG_PROC_DAY_22,OTH_ICD9_CODE_22,OTH_SURG_PROC_CODE_23,OTH_SURG_PROC_DAY_23,OTH_ICD9_CODE_23,OTH_SURG_PROC_CODE_24,OTH_SURG_PROC_DAY_24,OTH_ICD9_CODE_24,MS_MDC,MS_DRG,MS_GROUPER_VERSION_NBR,MS_GROUPER_ERROR_CODE,APR_MDC,APR_DRG,RISK_MORTALITY,ILLNESS_SEVERITY,APR_GROUPER_VERSION_NBR,APR_GROUPER_ERROR_CODE,ATTENDING_PHYSICIAN_UNIF_ID,OPERATING_PHYSICIAN_UNIF_ID,ENCOUNTER_INDICATOR,CERT_STATUS,FILLER_SPACE,TARGET
884115,320136748870,2013Q3,838400,Memorial Hermann Rehab Hospital Katy,Elective,Transfer from a hospital,R,NaN,NaN,NaN,NaN,Texas,77095,US,201,06,07,Female,White,Not of Hispanic,2,1,20,MA,MB,111,1671.00,0.00,1145.00,0.00,526.00,0.00,X,V5789,V5789,NaN,1919,Y,5119,Y,V8543,NaN,78459,Y,27801,Y,7812,Y,5180,Y,34590,Y,4019,Y,7993,Y,2724,Y,72887,Y,36250,Y,78093,Y,36901,Y,V5878,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

# Model Building

In [60]:
target = 'TARGET'

cat_features = ["TYPE_OF_ADMISSION","SOURCE_OF_ADMISSION","PAT_STATE", "RACE","SEX_CODE","TYPE_OF_BILL"]
num_features = ["LENGTH_OF_STAY","ILLNESS_SEVERITY","RISK_MORTALITY","APR_DRG","APR_MDC","ENCOUNTER_INDICATOR","CERT_STATUS","TOTAL_CHARGES_ANCIL","TOTAL_NON_COV_CHARGES_ANCIL","TOTAL_NON_COV_CHARGES_ACCOMM","TOTAL_CHARGES_ACCOMM","TOTAL_NON_COV_CHARGES","TOTAL_CHARGES"]
features = cat_features + num_features

print(f"{cat_features = }")
print(f"{num_features = }")
print(f"{features = }")

cat_features = ['TYPE_OF_ADMISSION', 'SOURCE_OF_ADMISSION', 'PAT_STATE', 'RACE', 'SEX_CODE', 'TYPE_OF_BILL']
num_features = ['LENGTH_OF_STAY', 'ILLNESS_SEVERITY', 'RISK_MORTALITY', 'APR_DRG', 'APR_MDC', 'ENCOUNTER_INDICATOR', 'CERT_STATUS', 'TOTAL_CHARGES_ANCIL', 'TOTAL_NON_COV_CHARGES_ANCIL', 'TOTAL_NON_COV_CHARGES_ACCOMM', 'TOTAL_CHARGES_ACCOMM', 'TOTAL_NON_COV_CHARGES', 'TOTAL_CHARGES']
features = ['TYPE_OF_ADMISSION', 'SOURCE_OF_ADMISSION', 'PAT_STATE', 'RACE', 'SEX_CODE', 'TYPE_OF_BILL', 'LENGTH_OF_STAY', 'ILLNESS_SEVERITY', 'RISK_MORTALITY', 'APR_DRG', 'APR_MDC', 'ENCOUNTER_INDICATOR', 'CERT_STATUS', 'TOTAL_CHARGES_ANCIL', 'TOTAL_NON_COV_CHARGES_ANCIL', 'TOTAL_NON_COV_CHARGES_ACCOMM', 'TOTAL_CHARGES_ACCOMM', 'TOTAL_NON_COV_CHARGES', 'TOTAL_CHARGES']


In [61]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2,SelectPercentile
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, recall_score

In [69]:
cat_preprocessor = Pipeline (
    steps = [
        ('impute',SimpleImputer(strategy='most_frequent')),
        ('encode', OneHotEncoder(handle_unknown="ignore")),
        ('select', SelectPercentile(chi2,percentile=80)),    
])

num_preprocessor = Pipeline (
    steps = [
        ('impute',SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),    
])

preprocessor = ColumnTransformer(
    transformers = [
        ('cat', cat_preprocessor, cat_features),
        ('num', num_preprocessor, num_features), 
])

model = Pipeline (
    steps = [
        ('pre', preprocessor),
        ('clf', RandomForestClassifier()),
])

In [70]:
X = df[features]
y = df[target]

In [71]:
X_train,X_test,y_train,y_test = train_test_split(X, y, stratify=y, test_size=0.4, random_state=SEED)
X_train.shape, X_test.shape

((599818, 19), (399880, 19))

In [72]:
model.fit(X_train, y_train)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('select',
                                                                   SelectPercentile(percentile=80,
                                                                                    score_func=<function chi2 at 0x000001F0EDE79AB0>))]),
                                                  ['TYPE_OF_ADMISSION',
                                                   'SOURCE_OF_ADMISSION',
                                                   'PAT_STATE', 'RACE',
                                                   'SEX_CODE',
                                                   'TYPE_O...
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['LENGTH_OF_STAY',
                                                   'ILLNESS_SEVERITY',
                                                   'RISK_MORTALITY', 'APR_DRG',
                                                   'APR_MDC',
                                                   'ENCOUNTER_INDICATOR',
                                                   'CERT_STATUS',
                                                   'TOTAL_CHARGES_ANCIL',
                                                   'TOTAL_NON_COV_CHARGES_ANCIL',
                                                   'TOTAL_NON_COV_CHARGES_ACCOMM',
                                                   'TOTAL_CHARGES_ACCOMM',
                                                   'TOTAL_NON_COV_CHARGES',
                                                   'TOTAL_CHARGES'])])),
                ('clf', RandomForestClassifier())])

In [73]:
y_pred = model.predict(X_train)

In [74]:
print(classification_report(y_train, y_pred, digits=4))

              precision    recall  f1-score   support

        long     1.0000    1.0000    1.0000    122465
      medium     1.0000    1.0000    1.0000    228862
       short     1.0000    1.0000    1.0000    248491

    accuracy                         1.0000    599818
   macro avg     1.0000    1.0000    1.0000    599818
weighted avg     1.0000    1.0000    1.0000    599818



In [75]:
y_pred = model.predict(X_test)

In [76]:
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

        long     1.0000    1.0000    1.0000     81644
      medium     1.0000    1.0000    1.0000    152575
       short     1.0000    1.0000    1.0000    165661

    accuracy                         1.0000    399880
   macro avg     1.0000    1.0000    1.0000    399880
weighted avg     1.0000    1.0000    1.0000    399880



In [80]:
dump(model, f"{ROOT}/output/model.joblib")

['.//output/model.joblib']

# Make Assigment

In [83]:
import my_lib as ml
ml.make_assignment()

Creating archive: my_assignment.zip
	dashboard.py - Skipped (missing file)
	df_grading_pred.csv - Skipped (missing file)
	my_lib.py - OK
	03-Model.ipynb - OK
	02-EDA.ipynb - OK
	01-Import.ipynb - OK
